### 使用传统的chains

In [5]:
from langchain.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 初始化Ollama模型
llm = Ollama(model="qwen2.5:3b")  # 可以替换为您安装的任何模型，如"gemma:2b"、"mistral"等

# 创建对话记忆
memory = ConversationBufferMemory(return_messages=True)

# 创建对话链
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    # verbose=True
)

# 测试多轮对话
print("开始对话测试:")

# 第一轮对话
user_input = "你好！"
print(f"用户: {user_input}")
response = conversation.predict(input=user_input)
print(f"AI: {response}")

# 第二轮对话
user_input = "我的名字是小明"
print(f"用户: {user_input}")
response = conversation.predict(input=user_input)
print(f"AI: {response}")

# 第三轮对话 - 测试记忆功能
user_input = "我的名字是什么？"
print(f"用户: {user_input}")
response = conversation.predict(input=user_input)
print(f"AI: {response}")


开始对话测试:
用户: 你好！
AI: 你好！很高兴见到你。有什么问题或话题想要讨论吗？我可以帮你解答关于科技、文化、日常生活等方面的问题，或者我们一起聊聊最近的新闻和趣闻。如果你有任何疑问或是想了解的事情，都可以随时问我哦。
用户: 我的名字是小明
AI: 很高兴认识你，小明！请问有什么问题或话题想要讨论吗？无论是科技、文化、日常生活方面的问题，还是最近的新闻和趣闻，我都很乐意与你分享。如果你有任何疑问或是想了解的事情，都可以随时问我哦。
用户: 我的名字是什么？
AI: 你的名字是小明。如果你有其他问题或话题想要讨论，也可以随时告诉我哦！


### 各种记忆组件

In [8]:
# 高级多轮对话记忆示例

from langchain_ollama import OllamaLLM  # 使用新的推荐导入方式
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory

# 初始化Ollama模型
llm = OllamaLLM(model="qwen2.5:3b")

# 创建自定义提示模板
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是一个有记忆能力的智能助手。请记住用户告诉你的信息，并在后续对话中使用这些信息。"),
    MessagesPlaceholder(variable_name="history"),
    HumanMessage(content="{input}")
])

# 1. 标准对话缓冲记忆 - 保存所有对话历史
buffer_memory = ConversationBufferMemory(return_messages=True, memory_key="history")

# 2. 对话摘要记忆 - 保存对话摘要而不是完整历史
summary_memory = ConversationSummaryMemory(llm=llm, return_messages=True, memory_key="history")

# 3. 对话窗口记忆 - 只保留最近的k轮对话
window_memory = ConversationBufferWindowMemory(k=2, return_messages=True, memory_key="history")

# 创建对话链 - 选择一种记忆类型
memory_type = "buffer"  # 可选: "buffer", "summary", "window"

if memory_type == "buffer":
    memory = buffer_memory
elif memory_type == "summary":
    memory = summary_memory
else:
    memory = window_memory

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    # verbose=True
)

# 测试多轮对话
print(f"\n使用 {memory_type} 记忆类型进行对话测试:")

# 模拟多轮对话
conversations = [
    "我的名字叫小明",
    "我的名字是什么?",
]

for user_input in conversations:
    print(f"\n用户: {user_input}")
    response = conversation.predict(input=user_input)
    print(f"AI: {response}")
    
# 查看内存中存储的内容
print("\n记忆中存储的内容:")
print(memory.load_memory_variables({}))



使用 buffer 记忆类型进行对话测试:

用户: 我的名字叫小明
AI: 好的，我已经记住了您提供的信息。您可以继续与我交流，我会记得之前的信息来帮助我们更好地互动。无论是聊天、提供信息还是其他任何你能想到的事情，请随时告诉我！

用户: 我的名字是什么?
AI: 你好，小明！很高兴认识你。你可以开始和我分享你的想法或者问题，我会尽力帮助你。

记忆中存储的内容:
{'history': [HumanMessage(content='我的名字叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='好的，我已经记住了您提供的信息。您可以继续与我交流，我会记得之前的信息来帮助我们更好地互动。无论是聊天、提供信息还是其他任何你能想到的事情，请随时告诉我！', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的名字是什么?', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，小明！很高兴认识你。你可以开始和我分享你的想法或者问题，我会尽力帮助你。', additional_kwargs={}, response_metadata={})]}


In [ ]:
# FastAPI集成示例 - 多轮对话记忆

import os
import json
from typing import Dict, List, Any, Optional
from pydantic import BaseModel, Field
from fastapi import FastAPI, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# 数据模型
class Message(BaseModel):
    role: str
    content: str

class ChatRequest(BaseModel):
    user_id: str
    message: str
    conversation_id: Optional[str] = None

class ChatResponse(BaseModel):
    conversation_id: str
    message: str
    history: List[Message]

# 内存中的会话存储
conversations = {}

# 创建一个简单的会话管理器
class ConversationManager:
    def __init__(self, model_name: str = "qwen2.5:3b"):
        self.model_name = model_name
        self.llm = OllamaLLM(model=model_name)
        self.conversations = {}
    
    def _get_or_create_conversation(self, conversation_id: str, user_id: str) -> Dict:
        """获取或创建会话"""
        if conversation_id not in self.conversations:
            memory = ConversationBufferMemory(return_messages=True, memory_key="history")
            
            prompt = ChatPromptTemplate.from_messages([
                SystemMessage(content="你是一个有记忆能力的智能助手。请记住用户告诉你的信息，并在后续对话中使用这些信息。"),
                MessagesPlaceholder(variable_name="history"),
                HumanMessage(content="{input}")
            ])
            
            conversation = LLMChain(
                llm=self.llm,
                prompt=prompt,
                memory=memory,
                verbose=True
            )
            
            self.conversations[conversation_id] = {
                "user_id": user_id,
                "conversation": conversation,
                "memory": memory,
                "messages": []
            }
        
        return self.conversations[conversation_id]
    
    def process_message(self, user_id: str, message: str, conversation_id: Optional[str] = None) -> Dict:
        """处理用户消息并返回响应"""
        # 如果没有提供会话ID，则创建一个新的
        if not conversation_id:
            conversation_id = f"conv_{len(self.conversations) + 1}"
        
        # 获取或创建会话
        conv_data = self._get_or_create_conversation(conversation_id, user_id)
        
        # 获取对话链
        conversation = conv_data["conversation"]
        
        # 获取AI响应
        response = conversation.predict(input=message)
        
        # 更新消息历史
        conv_data["messages"].append(Message(role="user", content=message))
        conv_data["messages"].append(Message(role="assistant", content=response))
        
        return {
            "conversation_id": conversation_id,
            "message": response,
            "history": conv_data["messages"]
        }
    
    def get_conversation_history(self, conversation_id: str) -> List[Message]:
        """获取会话历史"""
        if conversation_id not in self.conversations:
            return []
        
        return self.conversations[conversation_id]["messages"]
    
    def delete_conversation(self, conversation_id: str) -> bool:
        """删除会话"""
        if conversation_id in self.conversations:
            del self.conversations[conversation_id]
            return True
        return False

# 创建FastAPI应用和会话管理器实例
app = FastAPI(title="对话记忆API")
conversation_manager = ConversationManager()

# 添加CORS中间件
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# API端点
@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """处理聊天请求"""
    response = conversation_manager.process_message(
        user_id=request.user_id,
        message=request.message,
        conversation_id=request.conversation_id
    )
    return response

@app.get("/conversations/{conversation_id}")
async def get_conversation(conversation_id: str):
    """获取会话历史"""
    history = conversation_manager.get_conversation_history(conversation_id)
    if not history:
        raise HTTPException(status_code=404, detail="Conversation not found")
    return {"conversation_id": conversation_id, "history": history}

@app.delete("/conversations/{conversation_id}")
async def delete_conversation(conversation_id: str):
    """删除会话"""
    success = conversation_manager.delete_conversation(conversation_id)
    if not success:
        raise HTTPException(status_code=404, detail="Conversation not found")
    return {"message": "Conversation deleted successfully"}

# FastAPI应用启动代码
'''
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# 模拟API调用
print("\n模拟FastAPI对话记忆API调用:")

# 创建一个会话
user_id = "user123"
conversation_id = None

# 第一条消息
request1 = ChatRequest(user_id=user_id, message="你好，我是小明", conversation_id=conversation_id)
response1 = conversation_manager.process_message(
    user_id=request1.user_id,
    message=request1.message,
    conversation_id=request1.conversation_id
)
conversation_id = response1["conversation_id"]
print(f"\n用户: {request1.message}")
print(f"AI: {response1['message']}")

# 第二条消息
request2 = ChatRequest(user_id=user_id, message="我今年25岁，喜欢打篮球", conversation_id=conversation_id)
response2 = conversation_manager.process_message(
    user_id=request2.user_id,
    message=request2.message,
    conversation_id=request2.conversation_id
)
print(f"\n用户: {request2.message}")
print(f"AI: {response2['message']}")

# 第三条消息 - 测试记忆
request3 = ChatRequest(user_id=user_id, message="你还记得我的名字和爱好吗？", conversation_id=conversation_id)
response3 = conversation_manager.process_message(
    user_id=request3.user_id,
    message=request3.message,
    conversation_id=request3.conversation_id
)
print(f"\n用户: {request3.message}")
print(f"AI: {response3['message']}")

# 显示会话历史
print("\n会话历史:")
history = conversation_manager.get_conversation_history(conversation_id)
for msg in history:
    print(f"{msg.role.capitalize()}: {msg.content}")


# 多轮对话记忆功能总结

## 记忆类型比较

| 记忆类型 | 描述 | 适用场景 | 优点 | 缺点 |
|---------|------|---------|------|------|
| ConversationBufferMemory | 保存完整对话历史 | 短期对话，需要完整上下文 | 实现简单，保留所有信息 | 随着对话增长，上下文窗口可能溢出 |
| ConversationBufferWindowMemory | 只保留最近k轮对话 | 长期对话，但只关注最近上下文 | 控制上下文大小，减少token消耗 | 可能丢失早期重要信息 |
| ConversationSummaryMemory | 保存对话摘要而非完整历史 | 长期对话，需要压缩历史信息 | 大幅减少token消耗 | 可能丢失细节信息，依赖LLM摘要质量 |
| VectorStoreRetrieverMemory | 使用向量存储和检索相关记忆 | 长期对话，需要选择性记忆检索 | 可扩展性强，适合大规模对话 | 实现复杂，依赖嵌入质量 |
| EntityMemory | 专门记住实体信息 | 需要记住用户特定属性信息 | 结构化存储用户信息 | 需要额外的实体提取逻辑 |
| PersistentConversationMemory | 将对话历史保存到文件中 | 需要跨会话保持记忆 | 支持持久化，可恢复对话 | 需要文件I/O操作 |

## 最佳实践

1. **选择合适的记忆类型**：根据应用场景选择合适的记忆类型，或组合多种记忆类型
2. **优化提示模板**：在提示中明确指导模型如何使用记忆信息
3. **记忆管理**：实现记忆清理和过期机制，避免记忆过载
4. **持久化存储**：对重要对话实现持久化存储
5. **记忆压缩**：对长期记忆进行摘要或压缩
6. **相关性检索**：使用向量存储实现相关性记忆检索

## 实现步骤

1. 选择并初始化记忆组件
2. 创建适当的提示模板
3. 配置对话链或代理
4. 实现记忆管理逻辑
5. 处理用户输入并生成响应
6. 保存对话历史到记忆中

## 进阶功能

1. **多用户记忆隔离**：为不同用户维护独立的记忆空间
2. **记忆优先级**：对重要信息赋予更高的记忆优先级
3. **记忆检索增强**：结合关键词和语义检索
4. **记忆反思**：让AI定期反思和总结记忆内容
5. **记忆可视化**：为用户提供记忆内容的可视化界面

## 注意事项

1. 注意隐私保护，不要存储敏感信息
2. 实现记忆过期和清理机制
3. 控制记忆大小，避免token消耗过大
4. 定期评估记忆质量和相关性
5. 提供用户控制记忆的选项（如删除特定记忆）


In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_ollama import OllamaLLM
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# 初始化模型
llm = OllamaLLM(model="qwen2.5:3b")

# 创建记忆组件
memory = ConversationBufferMemory(return_messages=True, memory_key="history")

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是一个有记忆能力的智能助手。请记住用户告诉你的信息，并在后续对话中使用这些信息。"),
    MessagesPlaceholder(variable_name="history"),
    HumanMessage(content="{input}")
])

# 使用LCEL构建对话链
chain = (
    RunnablePassthrough.assign(
        history=lambda _: memory.load_memory_variables({})["history"]
    )
    | prompt
    | llm
)

# 处理对话函数
def process_message(user_input):
    response = chain.invoke({"input": user_input})
    # 修复：直接使用response，不访问content属性
    memory.save_context({"input": user_input}, {"output": response})
    return response

# 测试对话
print("开始对话测试:")

# 第一轮对话
user_input = "你好，我叫小明"
print(f"用户: {user_input}")
response = process_message(user_input)
print(f"AI: {response}")

# 第二轮对话
user_input = "我今年25岁"
print(f"用户: {user_input}")
response = process_message(user_input)
print(f"AI: {response}")

# 第三轮对话 - 测试记忆功能
user_input = "你还记得我的名字吗？"
print(f"用户: {user_input}")
response = process_message(user_input)
print(f"AI: {response}")

开始对话测试:
用户: 你好，我叫小明


AttributeError: 'str' object has no attribute 'content'